# Day 4 - Google Search grounding with the Gemini API

Welcome back to the Kaggle 5-day Generative AI course!

In this optional notebook, you will use [Google Search](https://google.com/) results with the Gemini API in a technique called grounding, where the model is connected to verifiable sources of information. Using search grounding is similar to using the RAG system you implemented earlier in the week, but the Gemini API automates a lot of it for you. The model generates Google Search queries and invokes the searches automatically, retrieving relevant data from Google's index of the web and providing links to search suggestions that support the query, so your users can verify the sources.

## New in Gemini 2.0

Gemini 2.0 Flash provides a generous Google Search quota as part of the [free tier](https://ai.google.dev/pricing). If you switch models back to 1.5, you will need to [enable billing](https://aistudio.google.com/apikey) to use Grounding with Google Search, or you can [try it out in AI Studio](https://aistudio.google.com/). See the [earlier versions of this notebook](https://www.kaggle.com/code/markishere/day-4-google-search-grounding?scriptVersionId=207458162) for guidance. 

## Optional: Use Google AI Studio

If you wish to try out grounding with Google Search, follow this section to try it out using the AI Studio interface. Or skip ahead to the `API` section to try the feature here in your notebook.

### Open AI Studio

Start by going to [AI Studio](https://aistudio.google.com/prompts/new_chat). You should be in the "New chat" interface.

Search Grounding is best with `gemini-2.0-flash`, but try out `gemini-1.5-flash` too.

![New chat in AI Studio](https://storage.googleapis.com/generativeai-downloads/kaggle/ais-newchat.png)

### Ask a question

Now enter a prompt into the chat interface. Try asking something that is timely and might require recent information to answer, like a recent sport score. For this query, grounding will be **disabled** by default.

This screenshow shows the response for `What were the top halloween costumes this year?`. Every execution will be different but typically the model talks about 2023, and hedges its responses saying it doesn't have access to specific information resulting in a general comment, rather than specific answers.

![Sample question-answer pair without grounding](https://storage.googleapis.com/generativeai-downloads/kaggle/cricket-ungrounded.png)

### Enable grounding

On the right-hand sidebar, under the `Tools` section. Find and enable the `Grounding` option.

![Enable grounding button](https://storage.googleapis.com/generativeai-downloads/kaggle/enable-grounding.png)

Now re-run your question by hovering over the user prompt in the chat history, and pressing the Gemini ✨ icon to re-run your prompt.

![Re-run prompt button](https://storage.googleapis.com/generativeai-downloads/kaggle/re-run-button.png)

You should now see a response generated that references sources from Google Search.

![Response with grounded sources from Google!](https://storage.googleapis.com/generativeai-downloads/kaggle/cricket-grounded.png)


### Try your own queries

Explore this interface and try some other queries. Share what works well in the [Discord](https://discord.com/channels/1101210829807956100/1303438361117069363)! You can start from [this blank template](https://aistudio.google.com/app/prompts/1FZtxKLFZIJ1p_0rICu8K2CNIF1tkAnf4) that has search grounding enabled.

The remaining steps require an API key with billing enabled. They are not required to complete this course; if you have tried grounding in AI Studio you are done for this notebook.

### Install packages
Start by installing and importing the Gemini API Python SDK.

In [1]:
# Install the google-genai, google-adk and google-api-core
!uv pip install -qU google-genai google-adk google-api-core

### import packages

In [2]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

'1.11.0'

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

If the `GOOGLE_API_KEY` is not available through Kaggle secrets, prompt to enter the key using `getpass` 

In [3]:
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
except Exception:
    import getpass
    GOOGLE_API_KEY = getpass.getpass("Enter your Google API Key: ")

client = genai.Client(api_key=GOOGLE_API_KEY)

### Setup Automated retry

In [4]:
# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

### Define `model_name` to be invoked

In [8]:
model_name = 'gemini-2.0-flash-lite'
# gemini-2.5-pro-preview-03-25
# gemini-2.0-flash
# gemini-2.0-flash-lite
# gemini-1.5-flash
# gemini-1.5-flash-lite
# gemini-1.5-pro	

### Define `few_show_prompt`


In [23]:
few_show_prompt = """
         Hello, You are an NBA analyst with deep expertise in NBA statistics.
            1. You are required to answer questions about 2025 playoff teams
            2. If you are not able to find the information, respond with "I don't know
            3. Provides citations with source and links for the answer"
         """ 

### Test the `few_show_prompt` without using Grounding
Here we will append a `question` to the `few_show_prompt` 

In [24]:
question = "What are the 2025 NBA playoff teams?"

# Ask for information without search grounding.
response = client.models.generate_content(
    model=model_name,
    contents=few_show_prompt+question)

Markdown(response.text)

I don't know. As of today, November 2, 2023, the 2025 NBA playoff teams are yet to be determined. The 2024-2025 NBA season hasn't even started, and predicting playoff teams that far in advance is highly speculative due to player development, trades, injuries, and other unforeseen circumstances.



## Use search grounding

### Make a request

To enable search grounding, you specify it as a tool: `google_search`. Like other tools, this is supplied as a parameter in `GenerateContentConfig`, and can be passed to `generate_content` calls as well as `chats.create` (for all chat turns) or `chat.send_message` (for specific turns).

In [25]:
model_name = 'gemini-2.0-flash'

In [26]:
# And now re-run the same query with search grounding enabled.
config_with_google_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

def query_with_grounding():
    response = client.models.generate_content(
        model=model_name,
        contents=few_show_prompt+question,
        config=config_with_google_search,
    )
    return response.candidates[0]


rc = query_with_grounding()
Markdown(rc.content.parts[0].text)

The 2025 NBA playoffs are underway! Here's a look at the teams involved and some expert predictions:

**Teams in the 2025 Playoffs:**

Based on the search results, the following teams are participating in the 2025 NBA Playoffs \[1]:

*   **Eastern Conference:**
    *   Boston Celtics
    *   Cleveland Cavaliers
    *   New York Knicks
    *   Milwaukee Bucks
    *   Indiana Pacers
    *   Detroit Pistons
    *   Orlando Magic
    *   Miami Heat
*   **Western Conference:**
    *   Oklahoma City Thunder
    *   Houston Rockets
    *   Los Angeles Lakers
    *   Denver Nuggets
    *   Los Angeles Clippers
    *   Minnesota Timberwolves
    *   Golden State Warriors
    *   Memphis Grizzlies

**First Round Matchups:**

Several sources mention specific first-round matchups \[2, 6]:

*   Oklahoma City Thunder vs. Memphis Grizzlies
*   Boston Celtics vs. Orlando Magic
*   Cleveland Cavaliers vs. Miami Heat
*   Houston Rockets vs. Golden State Warriors
*   Los Angeles Lakers vs. Minnesota Timberwolves
*   Los Angeles Clippers vs. Denver Nuggets
*   Milwaukee Bucks vs. Indiana Pacers
*   New York Knicks vs. Detroit Pistons

**Expert Predictions**

Many experts predict the Oklahoma City Thunder will win the NBA Finals over the Boston Celtics \[4]. One source suggests an upset with the Detroit Pistons beating the New York Knicks \[2].


In [15]:
from google.adk.agents import LlmAgent
from google.adk.tools import google_search

In [18]:
# Define the citations function declaration for the model
citations_function = {
    "name": "citations_for_answer",
    "description": "Provides citations for the answer.",
     "parameters": {
         "type": "object",
         "properties": {
             "source": {
                 "type": "string",
                 "description": "source of the information",
             },
             "datetime": {
                 "type": "string",
                 "description": "Date and Time of the published information.",
             },
            "url": {
                "type": "string",
                "description": "URL of the source.",
            },
         },
         "required": ["source", "datetime", "url"],
     },
 }

In [19]:
tools = types.Tool(function_declarations=[citations_function])
config = types.GenerateContentConfig(tools=[tools])

In [21]:
search_agent = LlmAgent(
    model=model_name,
    name='search_agent',
    instruction=few_show_prompt+question,
    tools=[google_search],
    output_key="data"
)
citation_prompt = "You're an expert in providing citations for the answer, Process data from state key 'data'."
citation_agent = LlmAgent(
    model=model_name,
    name='citation_agent',
    instruction=citation_prompt,
)
main_agent = LlmAgent(
    name="main_agent",
    model=model_name,
    instruction="You're an agent orchestrator, you can use your agents to get to the end result and serve the user",
    description="Main Agent",
    sub_agents=[ # Assign sub_agents here
        search_agent,
        citation_agent
    ]    
    # tools=[agent_tool.AgentTool(agent=search_agent), agent_tool.AgentTool(agent=citation_agent)]
)

In [22]:
APP_NAME = "test_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

from google.adk.runners import Runner
from google.genai import types 
from google.adk.sessions import InMemorySessionService

# Session and Runner
session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=main_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
  content = types.Content(role='user', parts=[types.Part(text=few_show_prompt+question)])
  events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  for event in events:
      if event.is_final_response():
          final_response = event.content.parts[0].text
          print("Agent Response: ", final_response)

call_agent("callback example")

Exception in thread Thread-45 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/Users/pparava1/.local/share/uv/python/cpython-3.12.10-macos-aarch64-none/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/Users/pparava1/git-ws/learning/gcp/google-gemini-gen-ai-samples/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/Users/pparava1/.local/share/uv/python/cpython-3.12.10-macos-aarch64-none/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/pparava1/git-ws/learning/gcp/google-gemini-gen-ai-samples/.venv/lib/python3.12/site-packages/google/adk/runners.py", line 138, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/Users/pparava1/.local/share/uv/python/cpython-3.12.10-macos-aarch64-none/lib/python3.12/asyncio/runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^

### Use Function calling to request citations with links

Create function declaration for citations

In [ ]:
google-adk

In [ ]:
tools = [
    {'google_search': {}},
    {'function_declarations': [citations_function]} # not defined here.
]

config = types.LiveConnectConfig(
    response_modalities=["TEXT"],
    tools=tools
)

async with client.aio.live.connect(model=model_name, config=config) as session:
    await session.send_client_content(
        turns={
            "role": "user",
            "parts": few_show_prompt+question,
        },
        turn_complete=True,
    )

    async for response in session.receive():
        print(response.tool_call)

In [ ]:
while not rc.grounding_metadata.grounding_supports or not rc.grounding_metadata.grounding_chunks:
    # If incomplete grounding data was returned, retry.
    rc = query_with_grounding()

chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(f'{chunk.web.title}: {chunk.web.uri}')

In [ ]:
# Separate the configurations for google_search and function calling.
# Configuration for google_search only.
config_with_google_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
)

# Configuration for function calling only.
config_with_function_calling = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch()), types.Tool(function_declarations=[citations_function])],
)

# Use the appropriate configuration based on the requirement.
def query_with_function_calling():
    response = client.models.generate_content(
        model=model_name,
        contents=few_show_prompt+question,
        config=config_with_function_calling,
    )
    return response.candidates[0]

# Example usage for function calling.
rc = query_with_function_calling()
Markdown(rc.content.parts[0].text)

### Response metadata

When search grounding is used, the model returns extra metadata that includes links to search suggestions, supporting documents and information on how the supporting documents were used.

Each "grounding chunk" represents information retrieved from Google Search that was used in the grounded generation request. Following the URI will take you to the source.

As part of the response, there is a standalone styled HTML content block that you use to link back to relevant search suggestions related to the generation.

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

The `grounding_supports` in the metadata provide a way for you to correlate the grounding chunks used to the generated output text.

In [ ]:
from pprint import pprint

supports = rc.grounding_metadata.grounding_supports
for support in supports:
    pprint(support.to_json_dict())

These supports can be used to highlight text in the response, or build tables of footnotes.

In [ ]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        rc.content.parts[0].text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"{i}. [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

## Search with tools

In this example, you'll use enable the Google Search grounding tool and the code generation tool across two steps. In the first step, the model will use Google Search to find the requested information and then in the follow-up question, it generates code to plot the results.

This usage includes textual, visual and code parts, so first define a function to help visualise these.

In [ ]:
from IPython.display import display, Image, Markdown

def show_response(response):
    for p in response.candidates[0].content.parts:
        if p.text:
            display(Markdown(p.text))
        elif p.inline_data:
            display(Image(p.inline_data.data))
        else:
            print(p.to_json_dict())
    
        display(Markdown('----'))

Now start a chat asking for some information. Here you provide the Google Search tool so that the model can look up data from Google's Search index.

In [ ]:
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0,
)

chat = client.chats.create(model='gemini-2.0-flash')

response = chat.send_message(
    message=prompt,
    config=config_with_search,
)

show_response(response)

Continuing the chat, now ask the model to convert the data into a chart. The `code_execution` tool is able to generate code to draw charts, execute that code and return the image. You can see the executed code in the `executable_code` part of the response.

Combining results from Google Search with tools like live plotting can enable very powerful use cases that require very little code to run.

In [ ]:
config_with_code = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
    temperature=0.0,
)

response = chat.send_message(
    message="Now plot this as a seaborn chart. Break out the medals too.",
    config=config_with_code,
)

show_response(response)